# 02.- Feature Engineering

In [7]:
import pandas as pd
import numpy as np
import pickle as pkl
from sklearn import preprocessing

# from utils.plotting import candlestick_plot, local_maxima_minima_plot

## Raw Data Loading

In [17]:
# Open the pickle file containing raw benchmark data.
with open('../data/raw/benchmark_data.pkl', 'rb') as handle:
    bm = pkl.load(handle)

data = bm['ibex']
# data['date'] = data.index
# data = data.reset_index(drop=True)
data.head()

,close,high,low,open,vol
2000-01-03,11610.0,11881.8,11574.4,11846.6,60107000.0
2000-01-04,11206.6,11530.0,11159.8,11499.5,62539000.0
2000-01-05,10863.1,11068.1,10824.9,11206.6,68153000.0
2000-01-07,11102.4,11137.9,10882.7,10963.4,144207000.0
2000-01-10,11173.3,11364.3,11120.6,11363.8,133817000.0


In [18]:
window_size = 50

# Randomly select a starting index
start_index = np.random.randint(0, len(data) - window_size + 1)

# Select the consecutive window of rows
random_window = data.iloc[start_index:start_index + window_size]

In [29]:
def local_min_max(data: pd.DataFrame) -> tuple[pd.Series,
                                               np.ndarray,
                                               np.ndarray]:
    """
    Identifies local minima and maxima from the 'low' values
    of a given time series using polynomial fitting.

    Args:
        data (pd.DataFrame): A DataFrame containing a 'low'
        column that represents the lowest values (e.g., stock lows).

    Returns:
        tuple[pd.Series, np.ndarray, np.ndarray]: 
            - A Series of the normalized 'low' values.
            - Indices of the local minima.
            - Indices of the local maxima.
    """

    # Normalize the 'low' values of the data using min-max scaling, 
    # bringing values into the range [0, 1].
    data_scaled = pd.Series(
        preprocessing.minmax_scale(data['low']),
        index=data.index
    ).reset_index(drop=True)
    
    # Prepare the data for polynomial fitting. 'x_data' are the 
    # indices and 'y_data' are the scaled 'low' values.
    x_data = data_scaled.index.tolist()
    y_data = data_scaled

    # Create a smooth range for fitting
    x = np.linspace(0, max(x_data), max(x_data) + 1)  

    # Fit a 15th degree polynomial to the data. This degree captures 
    # local variations in the data.
    pol = np.polyfit(x_data, y_data, 15)
    # Evaluate the polynomial at the specified x values
    y_pol = np.polyval(pol, x)

    # Find local minima (where the second derivative changes 
    # sign from negative to positive)
    local_min = (
        np.diff(np.sign(np.diff(y_pol))) > 0
        ).nonzero()[0] + 1

    # Find local maxima (where the second derivative changes 
    # sign from positive to negative)
    local_max = (
        np.diff(np.sign(np.diff(y_pol))) < 0
        ).nonzero()[0] + 1

    # Return the normalized 'low' values, local minima, and 
    # local maxima
    return data_scaled, local_min, local_max


## Features

### Feature 1: Absolute Extrema Duration
This feature could highlight relative compression or elongation of trends. For instance:

1. **Rapid Changes**:
   A small time difference could hint at erratic or impulsive market behavior, potentially filtering out broader, more gradual trends that might not fit a double-bottom scenario.

2. **Structure of Support and Resistance**:
   Longer time differences between the extremes could indicate a more stable consolidation, aligning with the characteristics of a well-formed double bottom, where the two lows and the interim peak occur over a longer timeframe.


In [30]:
def absolute_extrema_duration(data: pd.DataFrame) -> float:
    """Calculate the number of days between the maximum 
    and minimum values in a time series.

    Args:
        data (pd.DataFrame): DataFrame containing OHLC data.

    Returns:
        float: Number of days between the highest and 
        lowest values divided by the window lengh.
        
        This normalization can make your model more robust 
        when handling different time windows or comparing 
        different patterns
    """
    # Get the indices of the maximum and minimum 
    # values directly
    max_index = data.index.get_loc(
        data.high.idxmax().strftime("%Y-%m-%d"))
    min_index = data.index.get_loc(
        data.low.idxmin().strftime("%Y-%m-%d"))

    # Calculate the difference in days directly 
    # and normalizes the result
    duration = abs(
        (max_index - min_index)
        ) / len(data)
    
    return duration


In [31]:
absolute_extrema_duration(random_window)

0.3

### Feature 2: Low Threshold Minima Count

1. **Capturing Potential "Bottoms"**:
   The double-bottom pattern is characterized by two distinct troughs. By counting local minima that fall below a threshold, this feature highlights regions where the price exhibits pronounced dips, which are critical to identifying the pattern.

2. **Indicator of Significant Downward Movements**:
   A higher count of minima below the threshold suggests strong downward pressures within the window, increasing the likelihood of the presence of one or more "bottoms." This metric helps focus the analysis on windows with sufficient depth for the pattern to exist.

3. **Normalization Across Windows**:
   Dividing the count by the length of the data window ensures that the feature is scale-invariant and comparable across windows of different sizes. This is essential for ensuring that the feature is meaningful in varying contexts, regardless of the specific duration of the window.

4. **Threshold as a Proxy for Pattern Shape**:
   The chosen threshold (e.g., 0.20) is a proxy for the depth of price movements. Adjusting this value can help fine-tune the feature to be sensitive to the specific characteristics of double bottoms in the dataset being analyzed.


In [32]:
def low_threshold_minima_count(data: pd.DataFrame) -> float:
    """
    Calculate the proportion of local minima below a specified 
    threshold within a time window.

    Args:
        data (pd.DataFrame): DataFrame containing OHLC data.

    Returns:
        float: Proportion of local minima below the 0.20 
        threshold relative to the data length.
    """
    # Scale the 'low' column and find local minima
    data_scaled, local_min, _ = local_min_max(data)
    
    # Extract the values of the local minima
    local_min_values = data_scaled.loc[local_min]

    # Count minima below the threshold and normalize 
    # by the data length
    count = local_min_values.loc[
        (local_min_values < 0.20)].count() / len(data)
    
    # Return the computed proportion
    return count

In [33]:
low_threshold_minima_count(random_window)

np.float64(0.04)